# Time Series Forecasting

This notebook shows how to implement sequence models in tensorflow. Mainly, it uses recurrent neural network to do forecasting. The example below is based on climate chage data. Give time series temperatures, we will try to find patterns and predict how hot will it be in the upcoming months.

As for the machine learning, the following concepts are covered:

- Data prerocessing
    - data normalization
    - train/test split
    - creating windows
    - creating labels
    - preventing sequence bias
- Neural Networks
    - convolutional layers
    - LSTM layers
    - fully-connected layers
    - optimal choice of learning rate
- Callbacks
- Time Series Forecasting

## Data

The notebook is based on Climate Change dataset from Berkely Earth. It contains earth surface temperature for multiple cities in the world. You will hence be able to develop the model for a city of your choice!
The time series start in 1750 and has one observation per month.

To start, [download the data](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data/download) and place it in your working directory. 

In [ ]:
import csv
import datetime

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from utils import (find_year_index, forecast_timeseries, preprocess_timeseries,
                   unpack_file)

In [ ]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices: 
    tf.config.experimental.set_memory_growth(device, True)

In [ ]:
DATA_FILE = "climate-change-earth-surface-temperature-data.zip"
DATA_DIR = "climate"

unpack_file(DATA_FILE, DATA_DIR)

DATA_FILE = DATA_DIR + "/GlobalLandTemperaturesByCity.csv"
CITY = "Zurich"

In [ ]:
# Data loading 
dates = []
temperatures = []

date_index = 0
temp_index = 1
city_index = 3

with open(DATA_FILE, "r") as file:

    reader = csv.reader(file, delimiter=",")
    next(reader)
    
    for row in reader:
        if row[city_index] == CITY:
            dates.append(row[date_index])
            temperatures.append(row[temp_index])
            
dates = [datetime.date.fromisoformat(date) for date in dates]
temperatures = [float(temp) if temp != "" else None for temp in temperatures]

In [ ]:
# Let's see what was the temperature in the past years. 
start, stop = find_year_index(dates, 1900)

plt.figure(figsize=(22,6))
plt.plot(dates[start:stop],temperatures[start:stop])
plt.title('Temperature variation in {}'.format(CITY))
plt.ylabel("Temperature [C]")
plt.grid(True)
plt.show()

It looks like we have some missing values at the beginning of our series. Let's find out where they are.

In [ ]:
where_none = [i for i, temp in enumerate(temperatures) if temp == None]
print("There are missing values at indices: {}".format(where_none))

Given that they appear at the beginning only, let's just cut them off.

In [ ]:
temperatures = temperatures[107:-1]
dates = dates[107:-1]

In [ ]:
# Let's normalize the data. We will split the data at SPLIT_TIME for train and test set.
# The normalization is based moments from test set only (!).

SPLIT_TIME = 2775

train_data = temperatures[:SPLIT_TIME]
train_mean = sum(train_data)/len(train_data)
squared_diff = [(x - train_mean)**2 for x in train_data]
train_std = sum(squared_diff)/(len(train_data)-1)

normalized_data = [(x-train_mean)/train_std for x in temperatures]

In [ ]:
# Now we need to preprocess the data. We will use rolling window to create input sequence and labels.

WINDOW_SIZE = 24
BATCH_SIZE = 2
SHUFFLE_BUFFER = 50

input_data = preprocess_timeseries(normalized_data[:SPLIT_TIME], WINDOW_SIZE, BATCH_SIZE, SHUFFLE_BUFFER)

## Recurrent Neural Network

Let's build a recurrent neural network.  We will formulate this probelm as multiclass classification. As the input, we will use a sequences of temperatures. As the output, we will use the last measurement in this sequence. Neural network will then learn trend and seasonality patterns. As a result, we will be able to forecast tomorrow's temperature. 

In [ ]:
# Model architecture

tf.keras.backend.clear_session()

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size = 5, padding="causal", activation="relu", input_shape=[None,1]),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dense(30, activation="relu"),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1),
])

model.summary()

To start, we will define learning rate scheduler. It will automatically try out different learning rates for us. 
We will then be able to compare them, and choose the optimal. Now training longer is much more meaningful!

In [ ]:
# Time to train our model. 

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-6 * 10**(epoch / 4))

model.compile(optimizer="adam", 
              metrics=["mae"], 
              loss=tf.keras.losses.Huber())

history = model.fit(input_data, 
                    epochs=15, 
                    callbacks = [lr_schedule])

In [ ]:
# Find optimal learning rate

plt.semilogx(history.history["lr"], history.history["loss"])
plt.ylabel("Log loss")
plt.xlabel("Learning rate")

plt.axis([1e-6, 1.5e-4, -0.01, 0.02])

In [ ]:
# Continue training with optimal learning rate

OPTIMAL_LR = 1e-5

model.compile(optimizer=tf.keras.optimizers.Adam(lr=OPTIMAL_LR), 
              metrics=["mae"], 
              loss=tf.keras.losses.Huber())

history = model.fit(input_data, 
                    epochs=50)

In [ ]:
# Let's evaluate model performance on test data.

forecast = forecast_timeseries(model, normalized_data, WINDOW_SIZE)
forecast = forecast[SPLIT_TIME - WINDOW_SIZE:-1, -1, 0]

mae = tf.keras.metrics.mean_absolute_error(forecast, normalized_data[SPLIT_TIME:]).numpy()
mse = tf.keras.metrics.mean_squared_error(forecast, normalized_data[SPLIT_TIME:]).numpy()

print("Mean absolute error: {}".format(mae))
print("Mean squared error: {}".format(mse))

In [ ]:
# How does our forecast compare to realized values?

renormalized_forecast = [f*train_std + train_mean for f in forecast]

plt.figure(figsize=(10, 6))

plt.plot(dates[SPLIT_TIME:], temperatures[SPLIT_TIME:])
plt.plot(dates[SPLIT_TIME:], renormalized_forecast)
plt.ylabel("Temperature")
plt.legend(["Actual", "Forecast"])
plt.grid(True)
plt.show()